<img src="img/automation_using_flows.png">

In this notebook we demonstrate how the Globus Flows service can be used to automate data management at scale. We demonstrate a flow that automates a common design pattern: moving data from one system to another and making the data accessible to collaborators. This flow is often needed to manage data coming from instruments, e.g., image files can be moved from local storage attached to a microscope to a high-performance storage system where they may be accessed by all members of the research project.

We will walk through the following tasks:
1. Authenticate with Globus and get tokens for accessing various services
1. Define and register a flow with Globus
1. Execute a flow using configurable inputs for the endpoint and access permissions

The Globus flow is illustrated below.

<img src="img/transfer_set_permissions_flow.png" alt="Transfer and set permissions flow" align="CENTER" style="width: 90%;"/>

In [ ]:
import sys
import os
import time
import json
import uuid

from globus_automate_client import create_flows_client

CLIENT_ID = 'e369806e-2351-414e-8ce9-ff49370defd3'
flows_client = create_flows_client(CLIENT_ID)

# Feel free to replace the endpoint UUIDs below with those of your own endpoints
tutorial_endpoint = "ddb59aef-6d04-11e5-ba46-22000b92c6ec"  # endpoint "Globus Tutorial Endpoint 1"
petrel_endpoint = "e56c36e4-1063-11e6-a747-22000bf2d559"  # endpoint "Petrel Testbed"
tutorial_users_group = "50b6a29c-63ac-11e4-8062-22000ab68755"  # group "Tutorial Users"

## A. Authentication and Authorization

All interactions between users and services on the Globus automation platform are governed by the Globus Auth service. In particular, this means that consent must be given by the user for each interaction taking place on their part, including in this Notebook.

The first time you interact with each service such as the Flows service, or even a flow instance, you will be provided a link to perform the consent flow. You must click the link to complete the consent flow which will launch in a new tab. When complete, copy the code string, return to the notebook, and  paste the code into the input box that is presented below the link to begin the flow.

We will encounter authorization steps in a couple of places:
1. When deploying a new flow on the Globus Flows service; deplpoying a flow requires (a) an identity that is associated with a Globus subscription, and (b) access to the Flow service scope.
1. When executing a flow.

Access to the Flow service is already granted to you by virtue of authenticating to the JupyterHub running this notebook. Note: If you're running this notebook in your own environment you will need to manually log into Globus Auth and get tokens using a native app authorization flow (see the `Platform_Introduction_Native_App_Auth` notebook for an example of how to initiate this flow).

In [ ]:
# WILL FIX IN A MOMENT!

# import pickle, base64, os

# # Get Globus Auth token data from the JupyterHub environment
# data = pickle.loads(base64.b64decode(os.getenv('GLOBUS_DATA')))
    
# # Extract access token for each service
# transfer_token = data['tokens']['transfer.api.globus.org']['access_token']
# flow_token = ...
# ...other tokens??

# # Introspect the tokens
# print("YOUR TOKENS")
# print(f"Transfer: {transfer_token}")
# print(f"Flow: {flow_token}")
# print(f"....{other_token}")

# B. Flow Registration

## Define a flow

* Flows are composed of *Action* invocations.
* Each Action invocation reads from and contributes back to the *Flow State* which is referenced in Flow steps using the `InputPath` and `ResultPath` properties of an Action.
* Actions specify the service endpoint that will be called using the `ActionUrl` property, and the Globus Auth scope that's required for the specified action using the `ActionScope` property.
* Actions are linked via their `Next` property; the last action in a flow sets the `End` property to true.

Our simple flow defines just two *Actions*, `MoveFiles` and `SetPermission`.

In [ ]:


flow_definition = {
  "Comment": "Move files to guest collection and set access permissions",
  "StartAt": "MoveFiles",
  "States": {
    "MoveFiles": {
      "Comment": "Transfer from Globus Tutorial Endpoint 1 to a guest collection on Petrel",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint_id", 
        "destination_endpoint_id.$": "$.input.destination_endpoint_id",
        "sync_level": "exists",
        "transfer_items": [
              {
                "source_path.$": "$.input.source_path",
                "destination_path.$": "$.input.destination_path",
                "recursive": True
              }
        ],
      },
      "ResultPath": "$.MoveFiles",
      "WaitTime": 3600,
      "End": True,
#       "Next": "SetPermission"
    }, 
#     "SetPermission": {
#       "Comment": "Grant read permission on the data to the Tutorial users group",
#       "Type": "Action",
#       "ActionUrl": "https://actions.automate.globus.org/transfer/set_permission",
#       "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/set_permission",
#       "...": "$.....",
#       ...
#       ...
#       "End": True
#     }
  }
}



## Deploy a flow

Before running a flow it must be deployed on the Globus Flows service. In addition to the flow definition we created above, you must provide a unique title for your flow when you deploy it. If deployment succeeds Globus returns an ID as a handle to the flow resource.

In [ ]:
# Create a Globus Flows client
flows_client = create_flows_client(CLIENT_ID)

# Deploy the flow
flow_title = f"Tutorial-Flow-{str(uuid.uuid4())}"   # generate a unique title
# flow = flows_client.update_flow(flow_id)
flow = flows_client.deploy_flow(flow_definition, title=flow_title)
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']

print(f"Successfully deployed flow (ID: {flow_id})")
print(f"Flow scope: {flow_scope}")

# C. Flow Execution

## Define flow input(s)

If your flow includes parameterized input properties you must provide values for those properties when running the flow. Like the flow definition, flow inputs are defined as a JSON document. You must provide a value for each input property in your flow (input properties are prefixed by `$.` (see flow definition above).

For the `MoveFiles` action we must specify source and destination collection IDs and source and destination paths. For the `SetPermissions` action we must specify the collection ID, the type of entity to which we're granting permission, the entity's ID, and the permission (read or read/write).

In [ ]:
flow_input = {
    "input": {
        "source_path": "/share/godata",
        "destination_path": "/nick/godata",
        "source_endpoint_id": tutorial_endpoint,
        "destination_endpoint_id": petrel_endpoint,
    }
}


# flow_input = {
#   "MoveFilesInput": {
#     "source_endpoint_id": tutorial_endpoint,
#     "destination_endpoint_id": petrel_endpoint,
#     "transfer_items": [
#       {
#         "source_path": "/share/godata/",
#         "destination_path": "...",
#         "recursive": False
#       }
#     ]
#   },
#   "SetPermissionInput": {
#     "..."
#   }
# }

## Run the flow

We're finally ready to run the flow. Note that you will be required to consent again.

In [ ]:
from pprint import pprint

flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
flow_action_id = flow_action['action_id']
flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')
while flow_status == 'ACTIVE':
    time.sleep(2)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')
pprint (flow_action.data)